In [18]:
# pip install langchain langchain-openai langchain-groq langchain-community requests pandas yfinance

import os
import requests
import langchain
import ast
import numpy as np
import time
import json
from langchain.chat_models import ChatHuggingFace
import yfinance as yf
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [2]:
# pip install dotenv

In [41]:
import pandas as pd

from dotenv import load_dotenv

load_dotenv('variables.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [37]:
openai_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    api_key=OPENAI_API_KEY)

In [5]:
groq_llm_llama3_8b = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0,
    api_key=GROQ_API_KEY)

In [6]:
groq_llm_llama3_70b = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0,
    api_key=GROQ_API_KEY)

In [38]:
# Function to select LLM based on user preference

def get_llm(provider="openai", model="default"):
    if provider.lower() == "openai":
        return openai_llm
    elif provider.lower() == "groq":
        if model.lower() == "llama3-70b":
            return groq_llm_llama3_70b
        else:
            return groq_llm_llama3_8b
    else:
        # Default to OpenAI if provider not recognized
        return groq_llm_llama3_70b

# Task 1: Implement Stock Price Lookup Tool

def get_stock_price(symbol:str):

    """
    Fetches the latest stock price for the given symbol.

    Args:
        symbol (str): Stock ticker symbol (e.g., 'AAPL')
       
    Returns:
        dict: Stock information including current price and daily change

    """
    try:
        if not isinstance(symbol, str) or len(symbol) > 5:
            return f"Invalid symbol format: {symbol}"

        stock = yf.Ticker(symbol)
        data = stock.history(period="1d")

        if data.empty:
            return {"error": f"No data found for symbol: {symbol}"}

        current_price = data['Close'].iloc[-1]
        return round(current_price, 2)

    except Exception as e:

        return f"Error retrieving stock price for {symbol}: {str(e)}"

# Create the stock price lookup tool

stock_price_tool = Tool(
    name="StockPriceLookup",
    func=get_stock_price,
    description="Useful for getting the current price of a stock. Input should be a valid stock ticker symbol.")

In [8]:
stock_price_tool('AMZN')

<ipython-input-8-1d8fad36e577>:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  stock_price_tool('AMZN')


184.87

In [9]:
# Task 2: Implement Portfolio Rebalancing Tool

def rebalance_portfolio(portfolio_input):
    import json, ast

    try:
        for _ in range(2):  # try parsing twice to handle double-encoded strings
            if isinstance(portfolio_input, str):
                try:
                    portfolio_input = json.loads(portfolio_input)
                except json.JSONDecodeError:
                    portfolio_input = ast.literal_eval(portfolio_input)

        if not isinstance(portfolio_input, dict) or not portfolio_input:
            return "Invalid portfolio input. Use format like: {'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}"

        total_weight = sum(portfolio_input.values())
        if abs(total_weight - 1.0) > 0.01:
            return f"Total allocation must sum to 1.0. Current total: {round(total_weight, 2)}"

        equal_weight = 1.0 / len(portfolio_input)
        recommendations = []

        for symbol, weight in portfolio_input.items():
            diff = round(weight - equal_weight, 4)
            if abs(diff) < 0.01:
                recommendations.append(f"{symbol}: No action needed (balanced).")
            elif diff > 0:
                recommendations.append(f"{symbol}: Overweight by {diff:.2%} — Suggest SELL.")
            else:
                recommendations.append(f"{symbol}: Underweight by {-diff:.2%} — Suggest BUY.")

        return "\n".join(recommendations)

    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"



# Create the portfolio rebalancing tool

rebalance_tool = Tool(
    name="PortfolioRebalancer",
    func=rebalance_portfolio,
    description="Analyzes a portfolio and suggests rebalancing actions. Input should be a dictionary mapping stock symbols to their current weight in the portfolio.")

In [10]:
# Task 3: Implement Market Trend Analysis Tool

def market_trend_analysis(input= None):

    """
    Fetches stock market index trends over the past week.
    Returns:

        str: Analysis of market trends
    """
    try:

        spy = yf.Ticker("SPY")
        data = spy.history(period="7d")

        if data.empty or len(data) < 2:
            return "Not enough data to analyze SPY market trends."

        # 2. Calculate key metrics
        start_price = data['Close'].iloc[0]
        end_price = data['Close'].iloc[-5]
        daily_returns = data['Close'].pct_change().dropna()
        return_pct = ((end_price - start_price) / start_price) * 100
        volatility = np.std(daily_returns) * 100  # standard deviation as % for readability

        # 3. Trend direction
        trend = "upward 📈" if return_pct > 0 else "downward 📉" if return_pct < 0 else "flat ➖"

        # 4. Return a summary
        return (
            f"Market Trend (S&P 500 via SPY) - Past 5 Trading Days:\n"
            f"• Trend Direction: {trend}\n"
            f"• 5-Day Return: {return_pct:.2f}%\n"
            f"• Volatility: {volatility:.2f}% (daily std dev)\n"
            f"• Start Price: ${start_price:.2f}, End Price: ${end_price:.2f}")

    except Exception as e:

        return f"Error analyzing market trends: {str(e)}"

# Create the market trend analysis tool
    
trend_tool = Tool(
    name="MarketTrendAnalyzer",
    func=market_trend_analysis, 
    description="Provides S&P 500 trend analysis using SPY ETF data. No input required.")


In [11]:
def generate_prompt(user_portfolio):
    return f"""
    You are a financial assistant with access to the following tools:

    1. MarketTrendAnalyzer:
    - Description: Use this to analyze the recent S&P 500 market trend. This function takes NO input. Call it with: MarketTrendAnalyzer

    2. PortfolioRebalancer:
    - Description: Use this to check if a portfolio is balanced and suggest rebalancing actions. This function takes a single argument: a stringified dictionary where keys are stock tickers and values are portfolio weights (must sum to ~1.0).
    - Example input: '{{"AAPL": 0.5, "TSLA": 0.3, "GOOGL": 0.2}}'
    - Call it like this: PortfolioRebalancer('{{"AAPL": 0.5, "TSLA": 0.3, "GOOGL": 0.2}}')

    3. StockPriceLookup:
    - Description: Use this to fetch the latest price of any stock. This function takes a single string argument representing the ticker symbol.
    - Example input: "AAPL"
    - Call it like this: StockPriceLookup("AAPL")

    Task:
    Step 1: Use PortfolioRebalancer to analyze the portfolio: {str(user_portfolio)}.
    Step 2: Use MarketTrendAnalyzer to determine the current market trend.
    Step 3: (Optional) If needed, use StockPriceLookup to support your analysis.
    Step 4: Based on the above, provide a final recommendation on whether the user should rebalance their portfolio now or wait.

    Analyze this portfolio and recommend changes:  {str(user_portfolio)}
    """


In [40]:
# Function to create and run an agent with the selected LLM

def create_and_run_agent(llm_provider="openai", llm_model="default", query=""):
    llm = get_llm(llm_provider, llm_model)
    tools = [stock_price_tool, rebalance_tool, trend_tool]
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,)
    return agent.run(query)

### **OPEN AI API DOESN'T WORK BECAUSE I'M OUT OF CREDITS**

In [13]:
def run_test_cases():
    user_portfolio_1 = {"AAPL": 0.50, "TSLA": 0.30, "GOOGL": 0.20} 
    user_portfolio_2 = {"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}
    prompt1 = generate_prompt(user_portfolio_1)
    prompt2= generate_prompt(user_portfolio_2)
    # Test with Groq LLaMA3-8B
    print("\nGroq LLaMA3-8B Results:")
    print("Portfolio 1:", create_and_run_agent("groq", "llama3-8b",  query=prompt1))
    time.sleep(2)
    print("Portfolio 2:", create_and_run_agent("groq", "llama3-8b", query = prompt2))
if __name__ == "__main__":

    run_test_cases()

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Groq LLaMA3-8B Results:
Thought: I need to analyze the portfolio to see if it's balanced and suggest rebalancing actions. I'll start by using the PortfolioRebalancer tool.

Action: PortfolioRebalancer
Action Input: '{"AAPL": 0.5, "TSLA": 0.3, "GOOGL": 0.2}'
Observation: AAPL: Overweight by 16.67% — Suggest SELL.
TSLA: Underweight by 3.33% — Suggest BUY.
GOOGL: Underweight by 13.33% — Suggest BUY.
Thought:Thought: Now that I have analyzed the portfolio, I need to determine the current market trend to see if it's favorable for rebalancing. I'll use the MarketTrendAnalyzer tool.

Action: MarketTrendAnalyzer
Action Input: None
Observation: Market Trend (S&P 500 via SPY) - Past 5 Trading Days:
• Trend Direction: downward 📉
• 5-Day Return: -6.02%
• Volatility: 5.31% (daily std dev)
• Start Price: $536.70, End Price: $504.38
Thought:Thought: The market trend is currently downward, which might not be favorable for rebalancing. However, I still need to consider the portfolio's current state an

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to analyze the portfolio to see if it's balanced and suggest rebalancing actions. I'll start by using the PortfolioRebalancer tool.

Action: PortfolioRebalancer
Action Input: '{"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}'
Observation: MSFT: No action needed (balanced).
NVDA: No action needed (balanced).
AMZN: No action needed (balanced).
META: No action needed (balanced).
Thought:Thought: Now that I've analyzed the portfolio using PortfolioRebalancer, I see that all stocks are balanced, so no rebalancing actions are needed. However, I still need to analyze the current market trend to determine if the portfolio should be rebalanced.

Action: MarketTrendAnalyzer
Action Input: None
Observation: Market Trend (S&P 500 via SPY) - Past 5 Trading Days:
• Trend Direction: downward 📉
• 5-Day Return: -6.02%
• Volatility: 5.31% (daily std dev)
• Start Price: $536.70, End Price: $504.38
Thought:Thought: The MarketTrendAnalyzer suggests that the S&P 500 has been trending 

In [14]:
def run_test_cases():
    user_portfolio_1 = {"AAPL": 0.50, "TSLA": 0.30, "GOOGL": 0.20} 
    user_portfolio_2 = {"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}
    prompt1 = generate_prompt(user_portfolio_1)
    prompt2= generate_prompt(user_portfolio_2)
    # Test with Groq LLaMA3-70B
    print("\nGroq LLaMA3-70B Results:")
    print("Portfolio 1:", create_and_run_agent("groq", "llama3-70b", query = prompt1))
    time.sleep(2)
    print("Portfolio 2:", create_and_run_agent("groq", "llama3-70b", query = prompt2))

if __name__ == "__main__":

    run_test_cases()

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Groq LLaMA3-70B Results:
Thought: I need to analyze the given portfolio and determine if it needs rebalancing. 

Action: PortfolioRebalancer
Action Input: '{"AAPL": 0.5, "TSLA": 0.3, "GOOGL": 0.2}'
Observation: AAPL: Overweight by 16.67% — Suggest SELL.
TSLA: Underweight by 3.33% — Suggest BUY.
GOOGL: Underweight by 13.33% — Suggest BUY.
Thought:Thought: The portfolio analysis suggests that AAPL is overweight and TSLA and GOOGL are underweight. I need to consider the current market trend to determine if rebalancing is necessary.

Action: MarketTrendAnalyzer
Action Input: None
Observation: Market Trend (S&P 500 via SPY) - Past 5 Trading Days:
• Trend Direction: downward 📉
• 5-Day Return: -6.02%
• Volatility: 5.31% (daily std dev)
• Start Price: $536.70, End Price: $504.38
Thought:Thought: The market trend is downward, which may impact the portfolio's performance. I need to consider this trend when deciding whether to rebalance the portfolio.

Action: None
Action Input: None
Observation

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to analyze the given portfolio and determine if it needs rebalancing. 

Action: PortfolioRebalancer
Action Input: '{"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}'
Observation: MSFT: No action needed (balanced).
NVDA: No action needed (balanced).
AMZN: No action needed (balanced).
META: No action needed (balanced).
Thought:Thought: The portfolio seems to be balanced, but I need to consider the current market trend to make a final recommendation.

Action: MarketTrendAnalyzer
Action Input: None
Observation: Market Trend (S&P 500 via SPY) - Past 5 Trading Days:
• Trend Direction: downward 📉
• 5-Day Return: -6.02%
• Volatility: 5.31% (daily std dev)
• Start Price: $536.70, End Price: $504.38
Thought:Thought: The market trend is downward, which may impact the portfolio's performance. I need to consider this trend in my final recommendation.

Action: None (no additional action needed)
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:Though